In [226]:
import os
import pandas as pd

In [227]:
os.chdir('C:/My Documents/Mokymai ir kt info/Python/10_Final_Project')
os.getcwd()

'C:\\My Documents\\Mokymai ir kt info\\Python\\10_Final_Project'

# Main GIS Table

In [228]:
read_gis_file = pd.read_excel('GIS_DB/CS_GIS_DATA.xlsx')
read_gis_file.to_csv('GIS_DB/CS_GIS_DATA.csv', index = None, header = True)
df_gis = pd.read_csv('GIS_DB/CS_GIS_DATA.csv', parse_dates=['Start', 'Finish'])

In [229]:
df_gis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Activity ID             112 non-null    object        
 1   Activity Name           112 non-null    object        
 2   Start                   108 non-null    datetime64[ns]
 3   Finish                  112 non-null    datetime64[ns]
 4   At Completion Duration  112 non-null    int64         
 5   CS ID                   112 non-null    object        
 6   Stages                  112 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.2+ KB


In [230]:
df_gis.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages
0,A1330160,Name 1,2020-01-01,2020-01-01,0,CS6992,SCO.ROA
1,A1330170,Name 2,2020-01-01,2020-01-01,0,CS6881,SCO.ROA


# Clean Weekly Data update table

In [231]:
read_gis_file_update = pd.read_excel('Input/CS_Update_01_11_2021.xlsx', sheet_name= 1)
read_gis_file_update.to_csv('Input/CS_Update_01_11_2021.csv', index = False, header = True)
df_gis_update = pd.read_csv('Input/CS_Update_01_11_2021.csv')

In [232]:
df_gis_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Activity ID             112 non-null    object
 1   Activity Name           112 non-null    object
 2   Start                   108 non-null    object
 3   Finish                  112 non-null    object
 4   At Completion Duration  112 non-null    object
 5   CS ID                   112 non-null    object
 6   Stages                  112 non-null    object
dtypes: object(7)
memory usage: 6.2+ KB


In [233]:
df_gis_update.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages
0,A1330160,Name 1,13-Sep-22,31-May-24,433d,CS6992,SCO.ROA
1,A1330170,Name 2,13-Sep-22 A,31-May-24,433d,CS6881,SCO.ROA


In [234]:
df_gis_update['Start'] = df_gis_update['Start'].str[:9]
df_gis_update['Finish'] = df_gis_update['Finish'].str[:9]
df_gis_update['At Completion Duration'] = df_gis_update['At Completion Duration'].str.replace('d','')

In [235]:
df_gis_update.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages
0,A1330160,Name 1,13-Sep-22,31-May-24,433,CS6992,SCO.ROA
1,A1330170,Name 2,13-Sep-22,31-May-24,433,CS6881,SCO.ROA


In [236]:
df_gis_update.to_csv('Input/CS_Update_01_11_2021.csv', index = False)

In [237]:
df_gis_update = pd.read_csv('Input/CS_Update_01_11_2021.csv', parse_dates=['Start', 'Finish'])
df_gis_update.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages
0,A1330160,Name 1,2022-09-13,2024-05-31,433,CS6992,SCO.ROA
1,A1330170,Name 2,2022-09-13,2024-05-31,433,CS6881,SCO.ROA


In [238]:
# df_gis_update.drop('Unnamed: 0', axis='columns', inplace=True)

In [239]:
# df_gis_update.head(2)

In [240]:
df_gis_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Activity ID             112 non-null    object        
 1   Activity Name           112 non-null    object        
 2   Start                   108 non-null    datetime64[ns]
 3   Finish                  112 non-null    datetime64[ns]
 4   At Completion Duration  112 non-null    int64         
 5   CS ID                   112 non-null    object        
 6   Stages                  112 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 6.2+ KB


# Compare CSV's

In [241]:
df_gis_update['act_cs_id'] = (df_gis_update['Activity ID'] +"/"+ df_gis_update['CS ID'])
df_gis['act_cs_id'] = (df_gis['Activity ID'] +"/"+ df_gis['CS ID'])

In [242]:
df_gis_update.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
0,A1330160,Name 1,2022-09-13,2024-05-31,433,CS6992,SCO.ROA,A1330160/CS6992
1,A1330170,Name 2,2022-09-13,2024-05-31,433,CS6881,SCO.ROA,A1330170/CS6881


In [243]:
df_gis.head(2)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
0,A1330160,Name 1,2020-01-01,2020-01-01,0,CS6992,SCO.ROA,A1330160/CS6992
1,A1330170,Name 2,2020-01-01,2020-01-01,0,CS6881,SCO.ROA,A1330170/CS6881


In [244]:
def patikra():
    updvsorg = df_gis_update[~df_gis_update.act_cs_id.isin(df_gis.act_cs_id)]
    orgvsupd = df_gis[~df_gis.act_cs_id.isin(df_gis_update.act_cs_id)]
    if df_gis_update.act_cs_id.nunique() == len(df_gis_update.act_cs_id):
        if updvsorg.empty and orgvsupd.empty:
            return f'ID kodai ir objektu skaicius abejose lentelese sutampa. Viso objektu yra {df_gis_update.act_cs_id.nunique()}. \
Duomenys tinkami atnaujinimui'
        elif len(updvsorg) > 0 and orgvsupd.empty:
            return f'ERROR: Rasti nauji objektai ({len(updvsorg)} vnt.), kuriu nera GIS DB. \n{updvsorg}'
        elif len(orgvsupd) > 0 and updvsorg.empty:
            return f'ERROR: Atnaujinamoje lenteleje truksta objektu esanciu GIS DB ({len(orgvsupd)} vnt.). \n{orgvsupd}'
        elif len(updvsorg) > 0 and len(orgvsupd) > 0:
            return f'ERROR: Galimos klaidos:Nnatitinka ID/atsirado nauju objektu/truksta objektu. \n \
Objektai, kuriu nera GIS DB:\n {updvsorg}\n Objektai, kuriu nera atnaujinamoje lenteleje \n{orgvsupd}'
    else:
        return 'ERROR: ID reiksmes negali kartotis'
print (patikra())

ID kodai ir objektu skaicius abejose lentelese sutampa. Viso objektu yra 112. Duomenys tinkami atnaujinimui


# Update GIS DB

In [245]:
df_gis.head(3)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
0,A1330160,Name 1,2020-01-01,2020-01-01,0,CS6992,SCO.ROA,A1330160/CS6992
1,A1330170,Name 2,2020-01-01,2020-01-01,0,CS6881,SCO.ROA,A1330170/CS6881
2,A1330180,Name 3,2020-01-01,2020-01-01,0,CS6733,SCO.ROA,A1330180/CS6733


In [246]:
df_gis_update.head(3)

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
0,A1330160,Name 1,2022-09-13,2024-05-31,433,CS6992,SCO.ROA,A1330160/CS6992
1,A1330170,Name 2,2022-09-13,2024-05-31,433,CS6881,SCO.ROA,A1330170/CS6881
2,A1330180,Name 3,2021-12-02,2023-08-22,9999,CS6733,SCO.ROA,A1330180/CS6733


In [247]:
import numpy as np

In [258]:
updated = df_gis.merge(df_gis_update, how='left', on=['act_cs_id'], suffixes=('', '_new'))
updated[['Start', 'Finish', 'At Completion Duration']] = np.where(pd.notnull(updated[['Start_new', 'Finish_new', 'At Completion Duration_new']]), updated[['Start_new', 'Finish_new', 'At Completion Duration_new']], updated[['Start_new', 'Finish_new', 'At Completion Duration_new']])
updated.drop(['Start_new', 'Finish_new', 'Activity ID_new', 'Activity Name_new', 'At Completion Duration_new', 'CS ID_new', 'Stages_new'], axis=1, inplace=True)

In [259]:
updated.loc[[6, 7, 8, 9, 10, 11, 12]]

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
6,A1330220,Name 7,2023-02-21,2024-11-08,433,CS6240,SCO.ROA,A1330220/CS6240
7,A1330230,Name 8,2022-11-03,2024-07-23,433,CS6166,SCO.ROA,A1330230/CS6166
8,A13315,Name 9,NaT,2027-12-22,0,CS6010,SHO,A13315/CS6010
9,A13320,Name 10,NaT,2029-02-23,0,CS8900,SHO,A13320/CS8900
10,A13325,Name 11,NaT,2027-08-31,0,CS6011,SHO,A13325/CS6011
11,A13330,Name 12,NaT,2029-12-04,0,CS7900,SHO,A13330/CS7900
12,A1339330,Name 13,2027-10-15,2027-11-18,25,CS8712,SCO.TA.CIS,A1339330/CS8712


In [276]:
updated.to_csv('Output/updated.csv', index = False, header = True)
updated.to_excel('Output/updated.xlsx', index = False, header = True)

In [273]:
read_new_excel = pd.read_excel('Output/updated.xlsx', sheet_name= 0)

read_new_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Activity ID             112 non-null    object        
 1   Activity Name           112 non-null    object        
 2   Start                   108 non-null    datetime64[ns]
 3   Finish                  112 non-null    datetime64[ns]
 4   At Completion Duration  112 non-null    int64         
 5   CS ID                   112 non-null    object        
 6   Stages                  112 non-null    object        
 7   act_cs_id               112 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 7.1+ KB


In [277]:
read_new_excel

,Activity ID,Activity Name,Start,Finish,At Completion Duration,CS ID,Stages,act_cs_id
0,A1330160,Name 1,2022-09-13,2024-05-31,433,CS6992,SCO.ROA,A1330160/CS6992
1,A1330170,Name 2,2022-09-13,2024-05-31,433,CS6881,SCO.ROA,A1330170/CS6881
2,A1330180,Name 3,2021-12-02,2023-08-22,9999,CS6733,SCO.ROA,A1330180/CS6733
3,A1330190,Name 4,2021-12-02,2023-08-22,433,CS6672,SCO.ROA,A1330190/CS6672
4,A1330200,Name 5,2023-01-04,2024-09-23,433,CS6563,SCO.ROA,A1330200/CS6563
...,...,...,...,...,...,...,...,...
107,A1344470,Name 108,2026-11-25,2027-03-12,63,CS7011,SCO.TA.CIS,A1344470/CS7011
108,A1344480,Name 109,2027-03-15,2027-06-09,63,CS7210,SCO.TA.CIS,A1344480/CS7210
109,A1344490,Name 110,2027-06-10,2027-09-06,63,CS7412,SCO.TA.CIS,A1344490/CS7412
110,A1344500,Name 111,2027-09-07,2027-12-03,63,CS7612,SCO.TA.CIS,A1344500/CS7612


In [250]:
#def patikra2(updvsorg, orgvsupd):
    updvsorg = df_gis_update[~df_gis_update.act_cs_id.isin(df_gis.act_cs_id)]
    orgvsupd = df_gis[~df_gis.act_cs_id.isin(df_gis_update.act_cs_id)]
    if df_gis_update.act_cs_id.nunique() == len(df_gis_update.act_cs_id):
        if updvsorg.empty and orgvsupd.empty:
            return f'ID kodai ir objektu skaicius abejose lentelese sutampa. Viso objektu yra {df_gis_update.act_cs_id.nunique()}. Duomenys tinkami atnaujinimui'
        elif len(updvsorg) > 0: # and orgvsupd.empty:
            if len(updvsorg) > 0 and len(orgvsupd) > 0:
                return f'ERROR: Neatitinka kodai esantys abejose lentelese. \n Atnaujinimu lentele:\n {updvsorg}\n GIS lentele \n{orgvsupd}'
            else:
                return f'ERROR: Rasti nauji objektai ({len(updvsorg)}). Detalus sarasas pateiktas zemiau \n{updvsorg}'
        elif len(orgvsupd) > 0 and updvsorg.empty:
            return f'ERROR: Truksta objektu ({len(orgvsupd)}). Detalus sarasas pateiktas zemiau \n{orgvsupd}'
        elif len(updvsorg) > 0 and len(orgvsupd) > 0:
            return f'ERROR: Neatitinka kodai esantys abejose lentelese. \n Atnaujinimu lentele:\n {updvsorg}\n GIS lentele \n{orgvsupd}'
    else:
        return 'ERROR: ID reiksmes dubliuojasi'

print (patikra2(updvsorg, orgvsupd))

IndentationError: unexpected indent (Temp/ipykernel_12116/2606455982.py, line 2)